Block 1: Imports and Setup

In [24]:
import tensorflow as tf
import pandas as pd
import os

# Suppress warnings for cleaner output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

print("Libraries loaded.")

Libraries loaded.


Block 2: Configuration
Put your file paths here. This makes it easy to change folders without scrolling through logic code.

In [25]:
# --- CONFIGURATION ---

# Define your models and their specific input sizes
models_config = {
    # Replace these paths with your actual .h5 file locations
    "Model_AffectNet": {"path": "./affectnet_model.keras", "size": 96},
    "Model_FER2013":   {"path": "./fer2013_model.keras",   "size": 96},
    "Model_CKPlus":    {"path": "./ckplus_model.keras",    "size": 96},
    "Initial_model": {"path": "./finetuned_expressdetect_best.keras", "size": 48},
}

# Define your test datasets paths
datasets_config = {
    # Replace with the path to the 'test' or 'valid' folder of each dataset
    "Test_AffectNet": "./Sorted_data/test",
    "Test_FER2013":   "./fer2013/test",
    "Test_CKPlus":    "./CK_Plus_Ready/test"
}

print("Configuration defined.")

Configuration defined.


Block 3: The "Simple" Evaluator Function
This helper function loads the data on-the-fly and runs the standard evaluate command.

In [26]:
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.vgg16 import preprocess_input

def evaluate_model_on_data(model_path, data_path, img_size):
    try:
        # A. Load the Model
        model = tf.keras.models.load_model(model_path)

        # B. Load the Dataset (Auto-resized to match the model)
        test_data = image_dataset_from_directory(
            data_path,
            image_size=(img_size, img_size),
            batch_size=32,
            label_mode='categorical',
            shuffle=False, # Standard for testing
            verbose=0
        )

        # C. Apply Preprocessing (VGG16 specific)
        # Note: If you trained WITHOUT this, comment this line out!
        #test_data = test_data.map(lambda x, y: (preprocess_input(x), y))

        # D. The "One-Liner" Evaluation
        # returns [loss, accuracy]
        results = model.evaluate(test_data, verbose=0)

        return results[1] # Return just the accuracy (e.g., 0.89)

    except Exception as e:
        # This catches errors like mismatching classes (7 vs 8)
        print(f"Error: {e}")
        return None

Block 4: Execution & Table
This runs the loops and prints your nice 3x3 table.

In [27]:
# Create an empty table
results = pd.DataFrame(index=models_config.keys(), columns=datasets_config.keys())

print("Starting Evaluation (this may take a minute)...")

for model_name, m_info in models_config.items():
    print(f"\nEvaluating {model_name}...", end="")

    for data_name, data_path in datasets_config.items():
        print(f"\n  -> vs {data_name}...", end=" ")

        accuracy = evaluate_model_on_data(
            m_info['path'],
            data_path,
            m_info['size']
        )

        if accuracy is not None:
            results.loc[model_name, data_name] = f"{accuracy:.1%}"
            print(f"Done ({accuracy:.1%})", end="")
        else:
            results.loc[model_name, data_name] = "Error"
            print("Failed", end="")

print("\n\n" + "="*30)
print("FINAL RESULTS TABLE")
print("="*30)
display(results)

Starting Evaluation (this may take a minute)...

Evaluating Model_AffectNet...
  -> vs Test_AffectNet... Done (70.0%)
  -> vs Test_FER2013... Done (44.0%)
  -> vs Test_CKPlus... Done (84.4%)
Evaluating Model_FER2013...
  -> vs Test_AffectNet... Done (29.0%)
  -> vs Test_FER2013... Done (65.3%)
  -> vs Test_CKPlus... Done (82.3%)
Evaluating Model_CKPlus...
  -> vs Test_AffectNet... Done (26.4%)
  -> vs Test_FER2013... Done (29.7%)
  -> vs Test_CKPlus... Done (89.6%)
Evaluating Initial_model...
  -> vs Test_AffectNet... Error: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 96, 96, 3), found shape=(None, 48, 48, 3)
Failed
  -> vs Test_FER2013... Error: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 96, 96, 3), found shape=(None, 48, 48, 3)
Failed
  -> vs Test_CKPlus... Error: Input 0 of layer "functional_1" is incompatible with the layer: expected shape=(None, 96, 96, 3), found shape=(None, 48, 48, 3)
Failed

FI

,Test_AffectNet,Test_FER2013,Test_CKPlus
Model_AffectNet,70.0%,44.0%,84.4%
Model_FER2013,29.0%,65.3%,82.3%
Model_CKPlus,26.4%,29.7%,89.6%
Initial_model,Error,Error,Error
